In [1]:
import sapscript as ss
from IPython.display import display

In [2]:
sap = ss.sap()

In [3]:
session = None

In [26]:
ORDERS_CREATED_FILE = 'orders_created.txt' #берем отсюда номера ZIMG заказов, по которым планируем делать возвраты
with open(ORDERS_CREATED_FILE, 'r') as file:
    orders_created = file.read().split('\n')
    # Убираем пустые строки из списка заказов 
    orders_created = list(filter(lambda x: x, orders_created))
orders_created
try:
    sap = ss.sap()
    session = sap.session()
    
    # Ждем, пока все заказы прогрузятся в SAP, делаем 5 попыток обновления с интервалом в 1 сек
    for i in range(5):
        grid = session.findById("wnd[0]/usr/cntlALV_0100/shellcont/shellcont/shell/shellcont[0]/shell")
        _orders = sap.read_alv(grid, tech_names=True, cols=['VBELN'])
        # Убираем пустые строки из списка заказов 
        _orders = list(filter(lambda x: x, _orders.values))
        
        # Обновляем экран, пока не прогрузятся заказы из Hybris
        if len(_orders) < len(orders_created):
            time.sleep(1)
            session.findById("wnd[0]/usr/cntlALV_0100/shellcont/shellcont/shell/shellcont[0]/shell").pressToolbarButton("REFRESH")
    
except:
    sap.error_print()
finally:
    session = None


[array(['5630056978'], dtype=object)]


In [27]:
orders_created

['45961804', '45961805']

In [43]:
#проставляем статус Отгружен и заказов
try:
    session = sap.session()
    sap.check_system('ERD')
    sap.run('va02')
    session.findById("wnd[0]/usr/ctxtVBAK-VBELN").text = "45961682"
    session.findById("wnd[0]").sendVKey(0)
    try:
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
    except:
        None
    session.findById("wnd[0]/mbar/menu[2]/menu[1]/menu[12]").select()
    session.findById("wnd[0]/usr/tabsTAXI_TABSTRIP_HEAD/tabpT\\11/ssubSUBSCREEN_BODY:SAPMV45A:4305/btnBT_KSTC").press()

    session.findById("wnd[0]/usr/tabsTABSTRIP_0300/tabpANWS/ssubSUBSCREEN:SAPLBSVA:0302/tblSAPLBSVATC_E/radJ_STMAINT-ANWS[0,1]").press()
    session.findById("wnd[0]/usr/tabsTABSTRIP_0300/tabpANWS/ssubSUBSCREEN:SAPLBSVA:0302/tblSAPLBSVATC_E/radJ_STMAINT-ANWS[0,1]").setFocus()
    session.findById("wnd[0]").sendVKey(0)
    session.findById("wnd[0]/tbar[0]/btn[3]").press()
    session.findById("wnd[0]/tbar[0]/btn[11]").press()
except:
    sap.error_print()
finally:
    session = None

Ошибка
(<class 'AttributeError'>, AttributeError('<unknown>.press',), <traceback object at 0x0000006AA0327E48>)


In [5]:
try:
    sap.check_system('ERT')
    session = sap.session()
    
    sap.run('zimk')
    session.findById("wnd[0]/usr/ctxtS_WRK-LOW").text = "2024"
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    
    cols = ['VBELN', 'ADDTI', 'INITS_GDS', 'NETWR', 'DDLTI']
    
    grid = session.findById("wnd[0]/usr/cntlALV_0100/shellcont/shellcont/shell/shellcont[0]/shell")
    
    table = sap.read_alv(grid, cols)
    
    table.drop(index = table[table['Номер заказа SAP']==''].index, inplace=True)
    
    if table['ИсходнСтоимТоваров'].sum() != 0:
        percent_complete = table['СкомплСтоимПоз'].sum()/table['ИсходнСтоимТоваров'].sum()*100
    else:
        percent_complete = 100
    print('Процент комплектации = {:.2f}'.format(percent_complete))
    
except:
    sap.error_print()
finally:
    session = None

Процент комплектации = 20.52


In [9]:
table_all = sap.read_alv(grid)

AttributeError: <unknown>.rowcount

In [17]:
grid.GetColumnTitles('BOLNR')[1]

'ТранспНакладная'

In [8]:
table_all

,BOLNR,CNT,ICON_DDLTI,OTASK,VSBED_TX,ADDTI,DDLTI,VBELN,ESTAT,BULKP,...,COLLECT_TM,VDATU,DATES,SHTOAGN_HM,AGENT,AGENT_TX,DLVTI,NETWR,INITS_GDS,ACOMPL
0,,0,@AG@,Разместить,Последняя миля,30.08.19 11:17:29,31.08.19 12:00:00,5630054773,E0014,,...,60,31.08.2019,03.09.2019,,175001218,"ООО ""ГетТаксиРус""",14:00:00,1299.0,2598.0,50.00
1,,0,@AG@,Разместить,Последняя миля,30.08.19 12:22:46,31.08.19 12:00:00,5630054776,E0014,,...,60,31.08.2019,03.09.2019,,175001218,"ООО ""ГетТаксиРус""",14:00:00,1299.0,2598.0,50.00
2,,0,@AG@,Разместить,Последняя миля,30.08.19 12:23:04,31.08.19 12:00:00,5630054778,E0014,,...,60,31.08.2019,03.09.2019,,175001218,"ООО ""ГетТаксиРус""",14:00:00,1299.0,2598.0,50.00
3,,0,@AG@,Разместить,Последняя миля,30.08.19 12:23:10,31.08.19 12:00:00,5630054779,E0007,,...,60,31.08.2019,03.09.2019,,175001218,"ООО ""ГетТаксиРус""",14:00:00,2598.0,2598.0,100.00
4,,0,@AG@,Разместить,Последняя миля,30.08.19 12:23:21,31.08.19 12:00:00,5630054780,E0007,,...,60,31.08.2019,03.09.2019,,175001218,"ООО ""ГетТаксиРус""",14:00:00,2598.0,2598.0,100.00
5,,0,@AG@,Собрать,Последняя миля,30.08.19 16:45:01,31.08.19 18:00:00,5630054832,E0001,,...,60,31.08.2019,03.09.2019,,175010140,ООО «Портал»,20:00:00,0.0,3897.0,0.00
6,,0,@AG@,Собрать,Последняя миля,30.08.19 16:46:31,31.08.19 18:00:00,5630054836,E0001,,...,60,31.08.2019,03.09.2019,,175010140,ООО «Портал»,20:00:00,0.0,3507.3,0.00
7,,0,@AG@,Собрать,Последняя миля,30.08.19 16:46:43,31.08.19 18:00:00,5630054844,E0001,,...,60,31.08.2019,03.09.2019,,175010140,ООО «Портал»,20:00:00,0.0,3897.0,0.00
8,,0,@AG@,Собрать,Последняя миля,30.08.19 16:47:41,31.08.19 18:00:00,5630054848,E0001,,...,60,31.08.2019,03.09.2019,,175010140,ООО «Портал»,20:00:00,0.0,3507.3,0.00
9,,0,@AG@,Собрать,Последняя миля,30.08.19 16:52:37,31.08.19 18:00:00,5630054856,E0001,,...,60,31.08.2019,03.09.2019,,175010140,ООО «Портал»,20:00:00,0.0,3897.0,0.00
